In [9]:
from bids import BIDSLayout
import os

In [1]:
data_csv = '/lustre06/project/6061841/mwang8/data/.tabular/ppmi/bagel.csv'

In [6]:
PD = "Parkinson's Disease"
HC = "Healthy Control"
COHORT = "COHORT_DEFINITION"
BASELINE = "BL"
VISIT = "visit"
ID = "bids_id"
SESSION = 'ses-BL'
MODALITY = 'anat'

import pandas as pd
df = pd.read_csv(data_csv)
df_hc = df[(df[COHORT] == HC) & (df[VISIT] == BASELINE)]

In [7]:
df_hc

,participant_id,bids_id,visit,DATE_OF_BIRTH,SEX,COHORT_DEFINITION,EDUCATION,AGE,PRIMARY_DIAGNOSIS,UPDRS_1,...,UPDRS_3_OFF,UPDRS_4,MOCA,BENTON_LINE_ORIENTATION,CLOCK_DRAWING,LETTER_NUMBER_SEQUENCING,MODIFIED_BOSTON_NAMING_TEST,MODIFIED_SEMANTIC_FLUENCY,SYMBOL_DIGIT_MODALITIES,UPSIT
11,100004,sub-100004,BL,05/1963,1,Healthy Control,NaN,57.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,100890,sub-100890,BL,01/1943,1,Healthy Control,16.0,78.2,17.0,1.0,...,2.0,NaN,NaN,NaN,NaN,NaN,59.0,NaN,NaN,NaN
113,100956,sub-100956,BL,04/1946,0,Healthy Control,15.0,75.2,17.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN
156,101039,sub-101039,BL,02/1958,0,Healthy Control,16.0,63.2,17.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN
223,101195,sub-101195,BL,08/1947,1,Healthy Control,20.0,73.8,17.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,59.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10476,4116,sub-4116,BL,10/1948,1,Healthy Control,16.0,64.5,NaN,0.0,...,0.0,NaN,NaN,26.0,NaN,14.0,NaN,10.0,50.00,NaN
10477,4116,sub-4116,BL,10/1948,1,Healthy Control,NaN,64.5,NaN,0.0,...,0.0,NaN,NaN,26.0,NaN,14.0,NaN,10.0,50.00,NaN
10505,4118,sub-4118,BL,04/1945,0,Healthy Control,16.0,68.1,NaN,1.0,...,0.0,NaN,NaN,24.0,NaN,11.0,NaN,12.0,50.00,NaN
10506,4118,sub-4118,BL,04/1945,0,Healthy Control,NaN,68.1,NaN,1.0,...,0.0,NaN,NaN,24.0,NaN,11.0,NaN,12.0,50.00,NaN


In [11]:
data_path = '/home/niusham/projects/rrg-jbpoline/mwang8/data/.bids/ppmi/'


In [12]:
layout = BIDSLayout(data_path)

In [ ]:
layout.get()


In [18]:
layout.get(extension='nii.gz', suffix='T1w', run='run-01', return_type='filename')

[]

In [ ]:
#why I can not use run and session

In [ ]:
import numpy
import math


def read_transfo(file_name):
    """Read a 4x4 transformation matrix produced by FSL.

    Args:
        file_name: file containing the transformation matrix

    Returns:
        numpy array containing the transformation matrix
    """
    transfo = numpy.zeros(shape=(4, 4))
    transfo_line = 0
    with open(file_name) as f:
        for line in f:
            line = line.strip()
            elements = line.split()
            assert (
                len(elements) == 4
            ), "Wrong format in transformation line: {0} (file name: {1})".format(
                line, file_name
            )
            for i in range(0, 4):
                transfo[transfo_line][i] = float(elements[i].strip().replace(";", ""))
            transfo_line = transfo_line + 1

    transfo_mat = numpy.matrix(
        [
            [transfo[0][0], transfo[0][1], transfo[0][2], transfo[0][3]],
            [transfo[1][0], transfo[1][1], transfo[1][2], transfo[1][3]],
            [transfo[2][0], transfo[2][1], transfo[2][2], transfo[2][3]],
            [0, 0, 0, 1],
        ]
    )

    return transfo_mat


def get_rot_angle_vec(rot_mat):
    """Return the rotation angle and vector from a transformation matrix.

    Args:
        rot_mat: 4x4 transformation matrix

    Returns:
        rotation angle, rotation vector
    """
    # See http://www.euclideanspace.com/maths/geometry/rotations/conversions/matrixToAngle/
    cos = (rot_mat.item(0, 0) + rot_mat.item(1, 1) + rot_mat.item(2, 2) - 1) / 2
    if cos > 1:
        print("Warning: cos is larger than 1: {0}".format(cos))
        cos = 1
    rot_angle = math.acos(cos)
    x = (rot_mat.item(2, 1) - rot_mat.item(1, 2)) / math.sqrt(
        (rot_mat.item(2, 1) - rot_mat.item(1, 2)) ** 2
        + (rot_mat.item(0, 2) - rot_mat.item(2, 0)) ** 2
        + (rot_mat.item(1, 0) - rot_mat.item(0, 1)) ** 2
    )
    y = (rot_mat.item(0, 2) - rot_mat.item(2, 0)) / math.sqrt(
        (rot_mat.item(2, 1) - rot_mat.item(1, 2)) ** 2
        + (rot_mat.item(0, 2) - rot_mat.item(2, 0)) ** 2
        + (rot_mat.item(1, 0) - rot_mat.item(0, 1)) ** 2
    )
    z = (rot_mat.item(1, 0) - rot_mat.item(0, 1)) / math.sqrt(
        (rot_mat.item(2, 1) - rot_mat.item(1, 2)) ** 2
        + (rot_mat.item(0, 2) - rot_mat.item(2, 0)) ** 2
        + (rot_mat.item(1, 0) - rot_mat.item(0, 1)) ** 2
    )
    rot_vec = numpy.array([x, y, z])
    return rot_angle, rot_vec


def get_rot_mat(transfo_mat):
    """Return the rotation matrix from a transformation matrix.

    Args:
        transfo_mat: 4x4 transformation matrix

    Returns:
        Rotation matrix
    """
    rot_mat = numpy.matrix(
        [
            [transfo_mat.item(0, 0), transfo_mat.item(0, 1), transfo_mat.item(0, 2)],
            [transfo_mat.item(1, 0), transfo_mat.item(1, 1), transfo_mat.item(1, 2)],
            [transfo_mat.item(2, 0), transfo_mat.item(2, 1), transfo_mat.item(2, 2)],
        ]
    )
    return rot_mat


def get_tr_vec(transfo_mat):
    """Return translation vector from transformation matrix

    Args:
        transfo_mat: 4x4 transformation matrix

    Returns:
        Translation vector
    """
    tr_vec = numpy.array(
        [transfo_mat.item(0, 3), transfo_mat.item(1, 3), transfo_mat.item(2, 3)]
    )
    return tr_vec


def get_transfo_vector(transfo_mat):
    """Return transformation vector from transformation matrix

    Args:
        transfo_mat: 4x4 transformation matrix

    Returns:
        Transformation vector [tx, ty, tz, rx, ry, rz]
    """
    tx, ty, tz = get_tr_vec(transfo_mat)
    rx, ry, rz = get_euler_angles(transfo_mat)
    return [tx, ty, tz, rx, ry, rz]


def get_euler_angles(transfo_mat):
    """Compute Euler angles from transformation matrix

    Args:
        transfo_mat: 4x4 transformation matrix

    Returns:
        Euler angles rx, ry, rz
    """
    # From http://nghiaho.com/?page_id=846
    rx = math.atan2(transfo_mat.item(2, 1), transfo_mat.item(2, 2))
    ry = math.atan2(
        -transfo_mat.item(2, 0),
        math.sqrt(transfo_mat.item(2, 1) ** 2 + transfo_mat.item(2, 2) ** 2),
    )
    rz = math.atan2(transfo_mat.item(1, 0), transfo_mat.item(0, 0))
    return rx, ry, rz


def get_transfo_mat(x):
    """Return transformation matrix from transformation veector

    Args:
        x: transformation vector [tx, ty, tz, rx, ry, rz]

    Returns:
        4x4 transformation matrix
    """
    tx, ty, tz, rx, ry, rz = x
    x = numpy.matrix([[1, 0, 0], [0, cos(rx), -sin(rx)], [0, sin(rx), cos(rx)]])
    y = numpy.matrix([[cos(ry), 0, sin(ry)], [0, 1, 0], [-sin(ry), 0, cos(ry)]])
    z = numpy.matrix([[cos(rz), -sin(rz), 0], [sin(rz), cos(rz), 0], [0, 0, 1]])
    r = x * y * z
    mat = numpy.matrix(
        [
            [r.item(0, 0), r.item(0, 1), r.item(0, 2), tx],
            [r.item(1, 0), r.item(1, 1), r.item(1, 2), ty],
            [r.item(2, 0), r.item(2, 1), r.item(2, 2), tz],
            [0, 0, 0, 1],
        ]
    )
    return mat


def compare_transfos(mat1, mat2):
    """Return residual transformation vector between input matrices

    Args:
        mat1: 4x4 transformation matrix
        mat2: 4x4 transformation matrix

    Returns:
        Transformation vector representing mat1.I*mat2
    """
    residual_mat = mat1.I * mat2
    return get_transfo_vector(residual_mat)

In [1]:
from fsl.transform import flirt, affine

In [2]:
a = flirt.readFlirt('./outputs/anat-12dofs/mca/1/sub-100012_ses-BL.mat')

In [3]:
scales, translation, rotation, shears = affine.decompose(a, shears=True, angles=True)

In [4]:
rotation

[-0.20257406925794813, -0.0026581344362216035, 0.011358571991810385]

In [8]:
b=read_transfo('./outputs/anat-12dofs/mca/1/sub-100012_ses-BL.mat')

In [20]:
get_rot_angle_vec(b)

(0.0, array([-0.99891975, -0.01468602,  0.04408697]))

In [21]:
rotation

array([[ 0.99993196, -0.01059131, -0.00488881],
       [ 0.01135829,  0.97949483,  0.20114888],
       [ 0.00265813, -0.20119072,  0.97954848]])

In [22]:
get_rot_mat(b)

matrix([[ 1.07875685, -0.00521756, -0.00295225],
        [ 0.01225366,  1.01388682,  0.18763552],
        [ 0.00286767, -0.20822625,  1.07666064]])

In [16]:
translation

array([ -4.90924202, -30.77407001, -19.41439206])

In [17]:
get_tr_vec(b)

array([ -4.90924202, -30.77407001, -19.41439206])

In [18]:
translation

array([ -4.90924202, -30.77407001, -19.41439206])

In [24]:
shears

array([ 0.00532551,  0.00189193, -0.03168444])